# User Encoder - VAE

In [86]:
import numpy as np
import itertools
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime, timedelta
from scipy import sparse
%matplotlib inline

sns.set_theme(style="ticks")

## Dataset

In [87]:
df_all = pd.read_csv('../data/Booking/booking_train_set.csv', 
                 dtype={"user_id": str, "city_id": str, 'affiliate_id': str,
                       'utrip_id': str},parse_dates=['checkin', 'checkout'])

df_all.head()

,Unnamed: 0,user_id,checkin,checkout,city_id,device_class,affiliate_id,booker_country,hotel_country,utrip_id
0,0,1006220,2016-04-09,2016-04-11,31114,desktop,384,Gondal,Gondal,1006220_1
1,1,1006220,2016-04-11,2016-04-12,39641,desktop,384,Gondal,Gondal,1006220_1
2,2,1006220,2016-04-12,2016-04-16,20232,desktop,384,Gondal,Glubbdubdrib,1006220_1
3,3,1006220,2016-04-16,2016-04-17,24144,desktop,384,Gondal,Gondal,1006220_1
4,4,1010293,2016-07-09,2016-07-10,5325,mobile,359,The Devilfire Empire,Cobra Island,1010293_1


## Geral User Features

In [88]:
sample_days = 500
test_days   = 30

# Split Data
max_timestamp        = df_all.checkout.max()
init_train_timestamp = max_timestamp - timedelta(days = sample_days)
init_test_timestamp  = max_timestamp - timedelta(days = test_days)

# TODO Garantir que o usuário fique com a sessão no train ou test
df_train = df_all[(df_all.checkout >= init_train_timestamp) & (df_all.checkout < init_test_timestamp)]

df_train['visit'] = 1
df_train.head()

/home/marlesson/anaconda3/envs/booking_challenge/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,Unnamed: 0,user_id,checkin,checkout,city_id,device_class,affiliate_id,booker_country,hotel_country,utrip_id,visit
0,0,1006220,2016-04-09,2016-04-11,31114,desktop,384,Gondal,Gondal,1006220_1,1
1,1,1006220,2016-04-11,2016-04-12,39641,desktop,384,Gondal,Gondal,1006220_1,1
2,2,1006220,2016-04-12,2016-04-16,20232,desktop,384,Gondal,Glubbdubdrib,1006220_1,1
3,3,1006220,2016-04-16,2016-04-17,24144,desktop,384,Gondal,Gondal,1006220_1,1
4,4,1010293,2016-07-09,2016-07-10,5325,mobile,359,The Devilfire Empire,Cobra Island,1010293_1,1


In [89]:
df_user = df_train#[['user_id', 'checkin', 'checkout', 'booker_country', 'device_class', 'utrip_id']]

df_user['start_trip_month'] = df_user['checkin'].dt.month
df_user['duration']  = (df_user['checkout'] - df_user['checkin']).dt.days

df_user.head()

/home/marlesson/anaconda3/envs/booking_challenge/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/marlesson/anaconda3/envs/booking_challenge/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Unnamed: 0,user_id,checkin,checkout,city_id,device_class,affiliate_id,booker_country,hotel_country,utrip_id,visit,start_trip_month,duration
0,0,1006220,2016-04-09,2016-04-11,31114,desktop,384,Gondal,Gondal,1006220_1,1,4,2
1,1,1006220,2016-04-11,2016-04-12,39641,desktop,384,Gondal,Gondal,1006220_1,1,4,1
2,2,1006220,2016-04-12,2016-04-16,20232,desktop,384,Gondal,Glubbdubdrib,1006220_1,1,4,4
3,3,1006220,2016-04-16,2016-04-17,24144,desktop,384,Gondal,Gondal,1006220_1,1,4,1
4,4,1010293,2016-07-09,2016-07-10,5325,mobile,359,The Devilfire Empire,Cobra Island,1010293_1,1,7,1


In [90]:
import scipy
def mode(x):
    return scipy.stats.mode(x)[0]

df_user_geral = df_user.groupby(['user_id']).agg(
    checkins_count=('checkin', 'count'),
    utrip_id_count=('utrip_id', pd.Series.nunique),
    duration_mean=('duration', 'mean'),
    mode_booker_country=('booker_country', mode),
    mode_device_class=('device_class', mode),
    mode_trip_month=('start_trip_month', mode),
    mode_city_id=('city_id', mode),
    mode_affiliate_id=('affiliate_id', mode),
)

df_user_geral.sort_index()

,checkins_count,utrip_id_count,duration_mean,mode_booker_country,mode_device_class,mode_trip_month,mode_city_id,mode_affiliate_id
user_id,,,,,,,,
1000027,4,1,2.000000,Elbonia,desktop,8,15626,7168
1000033,5,1,2.000000,Gondal,desktop,4,38677,384
1000045,7,1,1.571429,The Devilfire Empire,desktop,6,31817,359
100008,5,1,1.800000,Gondal,desktop,7,11306,8436
1000083,4,1,1.250000,The Devilfire Empire,mobile,6,14705,359
...,...,...,...,...,...,...,...,...
999776,4,1,1.000000,Gondal,desktop,3,17775,4541
999839,4,1,2.000000,The Devilfire Empire,mobile,8,8335,359
999842,4,1,1.000000,Gondal,desktop,5,24036,3894


In [91]:
df_user_geral = df_user_geral.reset_index()
df_user_geral['user_id']  = df_user_geral['user_id'].astype('int')

## Trip  User Features

In [92]:
df_trip = pd.read_csv('../output/booking/dataset/train_500_30_5.csv')
df_trip.head()

,utrip_id,user_id,user_features,count_unique_city,trip_size,start_trip,end_trip,checkin_list,checkout_list,days_since_2016_list,...,affiliate_id_list,booker_country_list,hotel_country_list,step_list,first_city_id,first_hotel_country,last_city_id,last_hotel_country,country_count,duration_sum
0,1000027_1,1000027,"[-0.06505580991506577, 0.6599699854850769, 0.5...",4,3,2016-08-13,2016-08-18,"['0', '0', '2016-08-13', '2016-08-14', '2016-0...","['0', '0', '2016-08-14', '2016-08-16', '2016-0...","[0, 0, 225, 226, 228]",...,"['0', '0', '7168', '7168', '7168']","['0', '0', 'Elbonia', 'Elbonia', 'Elbonia']","['0', '0', 'Gondal', 'Gondal', 'Gondal']","[0, 0, 1, 2, 3]",8183,Gondal,30628,Gondal,97503,5
1,1000033_1,1000033,"[-0.41572630405426025, -0.05840621143579483, -...",4,4,2016-04-09,2016-04-16,"['0', '2016-04-09', '2016-04-11', '2016-04-12'...","['0', '2016-04-11', '2016-04-12', '2016-04-14'...","[0, 99, 101, 102, 104]",...,"['0', '359', '384', '384', '384']","['0', 'Gondal', 'Gondal', 'Gondal', 'Gondal']","['0', 'Cobra Island', 'Cobra Island', 'Cobra I...","[0, 1, 2, 3, 4]",38677,Cobra Island,38677,Cobra Island,104908,7
2,1000045_1,1000045,"[-0.3443673849105835, -0.9910609722137451, -1....",7,6,2016-06-18,2016-06-28,"['2016-06-20', '2016-06-22', '2016-06-24', '20...","['2016-06-22', '2016-06-24', '2016-06-25', '20...","[171, 173, 175, 176, 177]",...,"['7974', '7974', '7974', '359', '359']","['The Devilfire Empire', 'The Devilfire Empire...","['Fook Island', 'Fook Island', 'Carpathia', 'C...","[2, 3, 4, 5, 6]",64876,Fook Island,36063,Gondal,18213,8
3,1000083_1,1000083,"[0.38146448135375977, -1.216643214225769, 1.80...",4,3,2016-06-13,2016-06-16,"['0', '0', '2016-06-13', '2016-06-14', '2016-0...","['0', '0', '2016-06-14', '2016-06-15', '2016-0...","[0, 0, 164, 165, 166]",...,"['0', '0', '359', '359', '359']","['0', '0', 'The Devilfire Empire', 'The Devilf...","['0', '0', 'Osterlich', 'Osterlich', 'Osterlich']","[0, 0, 1, 2, 3]",55990,Osterlich,36063,Gondal,14936,3
4,100008_1,100008,"[0.13192051649093628, -1.070617437362671, -0.2...",5,4,2016-07-18,2016-07-25,"['0', '2016-07-18', '2016-07-21', '2016-07-22'...","['0', '2016-07-21', '2016-07-22', '2016-07-23'...","[0, 199, 202, 203, 204]",...,"['0', '8436', '8436', '8436', '8436']","['0', 'Gondal', 'Gondal', 'Gondal', 'Gondal']","['0', 'Kamistan', 'Kamistan', 'Kamistan', 'Kam...","[0, 1, 2, 3, 4]",11306,Kamistan,65690,Kamistan,6712,7


In [93]:
df_trip.iloc[0]

utrip_id                                                        1000027_1
user_id                                                           1000027
user_features           [-0.06505580991506577, 0.6599699854850769, 0.5...
count_unique_city                                                       4
trip_size                                                               3
start_trip                                                     2016-08-13
end_trip                                                       2016-08-18
checkin_list            ['0', '0', '2016-08-13', '2016-08-14', '2016-0...
checkout_list           ['0', '0', '2016-08-14', '2016-08-16', '2016-0...
days_since_2016_list                                [0, 0, 225, 226, 228]
duration_list                                             [0, 0, 1, 2, 2]
city_id_list                         ['0', '0', '8183', '15626', '60902']
device_class_list             ['0', '0', 'desktop', 'desktop', 'desktop']
affiliate_id_list                     

In [94]:
df_user_trip = df_trip.groupby(['user_id']).agg(
    trip_duration_mean=('duration_sum', 'mean'),
    first_city_id_last_trip=('first_city_id', 'last'),
    first_hotel_country_last_trip=('first_hotel_country', 'last'),
    
)
df_user_trip = df_user_trip.reset_index()
df_user_trip['user_id'] = df_user_trip['user_id'].astype('int')

In [95]:
df_user_all = df_user_geral.merge(df_user_trip, how="left").fillna(0)
df_user_all['first_city_id_last_trip'] = df_user_all['first_city_id_last_trip'].astype('str')
df_user_all['first_hotel_country_last_trip'] = df_user_all['first_hotel_country_last_trip'].astype('str')
df_user_all

,user_id,checkins_count,utrip_id_count,duration_mean,mode_booker_country,mode_device_class,mode_trip_month,mode_city_id,mode_affiliate_id,trip_duration_mean,first_city_id_last_trip,first_hotel_country_last_trip
0,1000027,4,1,2.000000,Elbonia,desktop,8,15626,7168,3.500000,8183.0,Gondal
1,1000033,5,1,2.000000,Gondal,desktop,4,38677,384,4.800000,38677.0,Cobra Island
2,1000045,7,1,1.571429,The Devilfire Empire,desktop,6,31817,359,5.833333,64876.0,Fook Island
3,100008,5,1,1.800000,Gondal,desktop,7,11306,8436,5.200000,11306.0,Kamistan
4,1000083,4,1,1.250000,The Devilfire Empire,mobile,6,14705,359,2.250000,55990.0,Osterlich
...,...,...,...,...,...,...,...,...,...,...,...,...
197187,999776,4,1,1.000000,Gondal,desktop,3,17775,4541,2.250000,17775.0,Novistrana
197188,999839,4,1,2.000000,The Devilfire Empire,mobile,8,8335,359,4.250000,8335.0,Cobra Island
197189,999842,4,1,1.000000,Gondal,desktop,5,24036,3894,2.250000,51291.0,Glubbdubdrib
197190,999855,15,1,1.533333,Gondal,mobile,4,38509,359,0.000000,0.0,0


In [96]:
df_user_all.describe()

,user_id,checkins_count,utrip_id_count,duration_mean,mode_trip_month,trip_duration_mean
count,1.971920e+05,197192.000000,197192.000000,197192.000000,197192.000000,197192.000000
mean,3.127540e+06,5.789591,1.085977,1.797557,7.062457,4.468437
std,1.806614e+06,3.009015,0.363647,0.806648,2.746306,2.279822
min,2.900000e+01,1.000000,1.000000,1.000000,1.000000,0.000000
25%,1.562400e+06,4.000000,1.000000,1.235294,5.000000,3.000000
50%,3.124848e+06,5.000000,1.000000,1.600000,8.000000,4.166667
75%,4.692839e+06,6.000000,1.000000,2.190476,9.000000,5.750000
max,6.258087e+06,172.000000,20.000000,30.000000,12.000000,18.250000


In [97]:
df_user_all.to_csv('../output/booking/dataset/user_features_raw.csv')

## Transform Dataset

In [98]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [99]:
df = df_user_all.set_index('user_id')
df.head()

,checkins_count,utrip_id_count,duration_mean,mode_booker_country,mode_device_class,mode_trip_month,mode_city_id,mode_affiliate_id,trip_duration_mean,first_city_id_last_trip,first_hotel_country_last_trip
user_id,,,,,,,,,,,
1000027,4,1,2.000000,Elbonia,desktop,8,15626,7168,3.500000,8183.0,Gondal
1000033,5,1,2.000000,Gondal,desktop,4,38677,384,4.800000,38677.0,Cobra Island
1000045,7,1,1.571429,The Devilfire Empire,desktop,6,31817,359,5.833333,64876.0,Fook Island
100008,5,1,1.800000,Gondal,desktop,7,11306,8436,5.200000,11306.0,Kamistan
1000083,4,1,1.250000,The Devilfire Empire,mobile,6,14705,359,2.250000,55990.0,Osterlich


In [100]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 197192 entries, 1000027 to 999944
Data columns (total 11 columns):
checkins_count                   197192 non-null int64
utrip_id_count                   197192 non-null int64
duration_mean                    197192 non-null float64
mode_booker_country              197192 non-null object
mode_device_class                197192 non-null object
mode_trip_month                  197192 non-null int64
mode_city_id                     197192 non-null object
mode_affiliate_id                197192 non-null object
trip_duration_mean               197192 non-null float64
first_city_id_last_trip          197192 non-null object
first_hotel_country_last_trip    197192 non-null object
dtypes: float64(2), int64(3), object(6)
memory usage: 18.1+ MB


In [101]:
df.select_dtypes(include=['int64', 'float64']).columns

Index(['checkins_count', 'utrip_id_count', 'duration_mean', 'mode_trip_month',
       'trip_duration_mean'],
      dtype='object')

In [102]:
# determine categorical and numerical features
numerical_ix   = df.select_dtypes(include=['int64', 'float64']).columns
categorical_ix = df.select_dtypes(include=['object', 'bool']).columns

In [103]:
df[categorical_ix]

,mode_booker_country,mode_device_class,mode_city_id,mode_affiliate_id,first_city_id_last_trip,first_hotel_country_last_trip
user_id,,,,,,
1000027,Elbonia,desktop,15626,7168,8183.0,Gondal
1000033,Gondal,desktop,38677,384,38677.0,Cobra Island
1000045,The Devilfire Empire,desktop,31817,359,64876.0,Fook Island
100008,Gondal,desktop,11306,8436,11306.0,Kamistan
1000083,The Devilfire Empire,mobile,14705,359,55990.0,Osterlich
...,...,...,...,...,...,...
999776,Gondal,desktop,17775,4541,17775.0,Novistrana
999839,The Devilfire Empire,mobile,8335,359,8335.0,Cobra Island
999842,Gondal,desktop,24036,3894,51291.0,Glubbdubdrib


In [104]:
# define the data preparation for the columns
t = [
     ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_ix), 
     ('num', StandardScaler(), numerical_ix)
    ]

col_transform = ColumnTransformer(transformers=t)
df_transform  = col_transform.fit_transform(df)
df_transform

<197192x35277 sparse matrix of type '<class 'numpy.float64'>'
	with 2169112 stored elements in Compressed Sparse Row format>

In [105]:
from scipy import sparse

sparse.save_npz("../output/booking/dataset/train_user_features.npz", df_transform)
df.reset_index().to_csv('../output/booking/dataset/train_user_features.csv', index=False)

In [21]:
from scipy import sparse
sparse.save_npz("../output/booking/dataset/all_user_features.npz", df_transform)
df.reset_index().to_csv('../output/booking/dataset/all_user_features.csv', index=False)

### Load and Save

In [30]:
df.head()

,checkins_count,utrip_id_count,duration_mean,mode_booker_country,mode_device_class,mode_trip_month,mode_city_id,mode_affiliate_id,trip_duration_mean,first_city_id_last_trip,first_hotel_country_last_trip
user_id,,,,,,,,,,,
1000027,4,1,2.000000,Elbonia,desktop,8,15626,7168,3.500000,8183.0,Gondal
1000033,5,1,2.000000,Gondal,desktop,4,38677,384,4.800000,38677.0,Cobra Island
1000045,7,1,1.571429,The Devilfire Empire,desktop,6,31817,359,6.714286,64876.0,Fook Island
100008,5,1,1.800000,Gondal,desktop,7,11306,8436,5.200000,11306.0,Kamistan
1000083,4,1,1.250000,The Devilfire Empire,mobile,6,14705,359,2.250000,55990.0,Osterlich


In [50]:
np_user_features = np.load('../output/booking/dataset/all_user_features.npy')
df['user_features'] = np_user_features.tolist()

In [44]:
df.head()

,checkins_count,utrip_id_count,duration_mean,mode_booker_country,mode_device_class,mode_trip_month,mode_city_id,mode_affiliate_id,trip_duration_mean,first_city_id_last_trip,first_hotel_country_last_trip,features,user_features
user_id,,,,,,,,,,,,,
1000027,4,1,2.000000,Elbonia,desktop,8,15626,7168,3.500000,8183.0,Gondal,"[-0.06505580991506577, 0.6599699854850769, 0.5...","[-0.06505580991506577, 0.6599699854850769, 0.5..."
1000033,5,1,2.000000,Gondal,desktop,4,38677,384,4.800000,38677.0,Cobra Island,"[-0.41572630405426025, -0.05840621143579483, -...","[-0.41572630405426025, -0.05840621143579483, -..."
1000045,7,1,1.571429,The Devilfire Empire,desktop,6,31817,359,6.714286,64876.0,Fook Island,"[-0.3443673849105835, -0.9910609722137451, -1....","[-0.3443673849105835, -0.9910609722137451, -1...."
100008,5,1,1.800000,Gondal,desktop,7,11306,8436,5.200000,11306.0,Kamistan,"[0.13192051649093628, -1.070617437362671, -0.2...","[0.13192051649093628, -1.070617437362671, -0.2..."
1000083,4,1,1.250000,The Devilfire Empire,mobile,6,14705,359,2.250000,55990.0,Osterlich,"[0.38146448135375977, -1.216643214225769, 1.80...","[0.38146448135375977, -1.216643214225769, 1.80..."


In [45]:
df.reset_index().to_csv('../output/booking/dataset/all_user_features.csv')

In [52]:
from numpy import asarray
from numpy import savetxt

# save to csv file
savetxt('data.csv', np_user_features, delimiter='\t')

In [55]:
df.reset_index().to_csv('metadata.csv', sep='\t')